In [2]:
import requests
import pandas as pd
import time
from typing import Dict, List, Optional

In [ ]:
class Scraper:
    def __init__(self):
        self.base_url = "https://fbrapi.com"
        self.api_key = None
        self.premier_league_id = 9

    def generate_api_key(self) -> str:
        try:
            response = requests.post(f"{self.base_url}/generate_api_key")
            response.raise_for_status()
            api_key = response.json()['api_key']
            self.api_key = api_key
            print(f"API Key generated successfully: {api_key}")
            return api_key
        except requests.exceptions.RequestException as e:
            print(f"Error generating API key: {e}")
            return None
    
    def make_request(self, endpoint: str, params: Dict = None) -> Optional[Dict]:
        if not self.api_key:
            print("No API key available. Please generate one first")
            return None
        
        headers = {"X-API-KEY": self.api_key}
        url = f"{self.base_url}/{endpoint}"

        try:
            time.sleep(6)
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error making requests to {endpoint}: {e}")
            return None
        
    def get_available_seasons(self) -> List[str]:
        data = self.make_requests("league-seasons", {"league_id": self.premier_league_id})
        if data and 'data' in data:
            return [season['season_id'] for season in data['data']]
        return []
    
    def get_teams_for_seasons(self, season_id: str) -> List[Dict]:
        data = self.make_requests("league-standings", {
            "league_id": self.premier_league_id,
            "season_id": season_id
        })

        teams = []
        if data and 'data' in data:
            for standings_group in data['data']:
                if 'standings' in standings_group:
                    for team in standings_group['standings']:
                        teams.append({
                            'team_id': team['team_id'],
                            'team_name': team['team_name']
                        })
        
        return teams
    
    def get_team_matches(self, team_id: str, season_id: str) -> List[Dict]:
        data = self.make_requests("matches", {
            "team_id": team_id,
            "league_id": self.premier_league_id,
            "season_id": season_id
        })

        if data and 'data' in data:
            return data['data']
        return []
    
    def get_team_match_stats(self, team_id: str, season_id: str) -> List[Dict]:
        data = self.make_request("team-match-stats", {
            "team_id": team_id,
            "league_id": self.premier_league_id,
            "season_id": season_id,
        })

        if data and 'data' in data:
            return data['data']
        return []
    
    def process_team_data(self, team_id: str, team_name: str, season_id: str) -> pd.DataFrame:
        print(f"Processing {team_name} for season_id...")

        matches = self.get_team_matches(team_id, season_id)
        if not matches:
            print(f"No matches found for {team_name} in {season_id}")
            return pd.DataFrame()
        
        match_stats = self.get_team_match_stats(team_id, season_id)
        if not match_stats:
            print(f"No match stats found for {team_name} in {season_id}")
            return pd.DataFrame()
        
        matches_data = []
        for match in matches:
            matches_data.append({
                'Data': match['data'],
                'Time': match['time'],
                'Comp': 'Premier League',
                'Round': match['round'],
                'Venue': 'Home' if match['home_away'] == 'Home' else 'Away',
                'Result': match['result'],
                'GF': match['gf'],
                'GA': match['ga'],
                'Opponent': match['opponent'],
                'Formation': match.get('formation', ''),
                'Referee': match.get('referee', ''),
                'Match Report': '',
                'Notes': ''
            })

        matches_df = pd.DataFrame(matches_data)

        shooting_data = []
        for match_stat in match_stats:
            if 'stats' in match_ and 'shotting' in match_stat['stats']:
                shooting_stats = match_stat['stats']['shooting']
                shooting_data.append({
                    'Data': match_stat['meta_data']['date'],
                    'Sh': shooting_stats.get('sh', 0),
                    'SoT': shooting_stats.get('sot', 0),
                    'Dist': shooting_stats.get('avg_sh_dist', 0),
                    'FK': shooting_stats.get('fk_sh', 0),
                    'PK': shooting_stats.get('pk_made', 0)
                    'PKatt': match_stat['stats'].get('schedule', {}).get('pk_att', 0) if 'schedule' in match_stat['stats'] else 0
                })

        shooting_df = pd.DataFrame(shooting_data)

        